In [162]:
import pennylane as qml

In [163]:
n_wires = 3
anc1 = n_wires
anc2 = n_wires + 1
anc3 = n_wires + 2

In [164]:
dev_ideal = qml.device('default.mixed', wires=n_wires + 3 + 2)

In [165]:
from scipy.optimize import fsolve
from pennylane import numpy as np

# Define the expression whose roots we want to find

phi1 = 0.5
k = 2

def get_phi2(k, phi1):
    func = lambda phi2 : k - (np.cos(2*phi1 + phi2)/np.cos(2*phi1 - phi2)) 

    phi2_initial_guess = 0.5
    solution = fsolve(func, phi2_initial_guess)

    return solution

phi2 = get_phi2(k, phi1)

In [166]:
def U_a(wires):
    [qml.Hadamard(wires=i) for i in wires]

In [167]:
def c0not(ctrl, target):
    # cnot conditioned on 0
    qml.PauliX(wires=ctrl)
    qml.CNOT(wires=(ctrl, target))
    qml.PauliX(wires=ctrl)

In [168]:
def tunable_A(phi1, phi2):
    qml.Hadamard(wires=anc2)
    c0not(anc1, anc2)

    # Z rotation by phi1
    qml.RZ(phi1, wires=anc2)

    c0not(anc1, anc2)
    # U_a acts on all wires + first ancilla
    U_a(wires=range(anc1+1))
    c0not(anc1, anc2)

    # Z rotation by phi2
    qml.RZ(phi2, wires=anc2)

    c0not(anc1, anc2)
    U_a(wires=range(anc1+1)) # hadamards are hermititian so conjugate = itself
    c0not(anc1, anc2)

    # Z rotation by phi2
    qml.RZ(phi1, wires=anc2)
    
    c0not(anc1, anc2)
    qml.Hadamard(wires=anc2)

    # destroy the ancilla qubits
    # qml.measure(wires=anc1, reset=True)
    # qml.measure(wires=anc2, reset=True)


In [169]:
matrix_fn = qml.matrix(tunable_A)

In [170]:
matrix_fn(phi1, phi2).shape

(32, 32)

In [171]:
@qml.qnode(dev_ideal)
def circuit(phi):
    qml.RY(phi, wires = 0)
    qml.ControlledQubitUnitary(matrix_fn(np.pi/4, np.pi/4), control_wires=anc3, wires=range(anc2+1))
    # reset the ancilla qubits
    qml.measure(anc1, reset=True)
    qml.measure(anc2, reset=True)

    return qml.expval(qml.PauliZ(0))

In [172]:
print(qml.draw(circuit, expansion_strategy="device")(0))

0: ──RY(0.00)─╭U(M0)─────────────┤  <Z>
1: ───────────├U(M0)─────────────┤     
2: ───────────├U(M0)─────────────┤     
3: ───────────├U(M0)─╭●─╭X───────┤     
4: ───────────├U(M0)─│──│──╭●─╭X─┤     
5: ───────────╰●─────│──│──│──│──┤     
6: ──────────────────╰X─╰●─│──│──┤     
7: ────────────────────────╰X─╰●─┤     
M0 = 
[[ 6.53281482e-01-1.11259613e-17j  2.64361880e-18+4.07355913e-34j
  -1.10600424e-17-5.73584186e-20j ...  1.47404296e-34-7.56735282e-35j
   1.87765116e-18+4.53305090e-18j  6.10568587e-35+1.47404296e-34j]
 [ 2.64361880e-18+4.07355913e-34j  6.53281482e-01-1.11259613e-17j
  -2.08563115e-35+4.75237382e-37j ... -1.87765116e-18-1.01021999e-17j
   6.10568587e-35+1.47404296e-34j  1.87765116e-18+4.53305090e-18j]
 [-1.10600424e-17-5.73584186e-20j -2.08563115e-35+4.75237382e-37j
   6.53281482e-01-1.11259613e-17j ...  6.10568587e-35+1.47404296e-34j
  -1.87765116e-18-1.01021999e-17j  1.47404296e-34-7.56735282e-35j]
 ...
 [-3.40674538e-37-4.30716518e-34j  2.37586349e-20-5.62650738

In [173]:
from IPython.display import clear_output
import time

opt = qml.GradientDescentOptimizer(0.1)
w = 0.01 * np.random.randn(1)
cost_history = []

err = float("inf")
it = 1

# training loop

start = time.time()
for it in range(100):
    w, cost = opt.step_and_cost(circuit, w)
    
    clear_output(wait=True)

    print("Step {:3d}       Cost_L = {:9.7f} \t error = {:9.7f}".format(it, cost, err), flush=True)
    cost_history.append(cost)

    prev_w = w

    it += 1

TTS = time.time() - start

print(f"Training time: {TTS}s")

Step  99       Cost_L = -0.9995123 	 error =       inf
Training time: 26.33497905731201s
